In [1]:
# Exp-00
# this notebook defines the configurations to explore in Exp-00s
#  - the shared params
#  - the configs for classic
#  - the FGN variants of those classic models

In [10]:
from __future__ import print_function

In [9]:
import itertools
import random
import time, os
import pickle

In [14]:
# saved in :
save_dir =  "/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/"
# timestamp = time.strftime("%Y_%m_%d_at_%H:%M:%S")
# print(timestamp)
# try:
#     os.mkdir(save_dir + '/' + timestamp)
# except e:
#     pass

In [4]:
# shared configurations 

In [5]:
# fixed experiment parameters
batch_size = 168
in_feats = 28*28
out_feats = 10
num_epochs = 5
# lmbda_l2 = (4.0*0.1/len(mnist_train_loader.dataset))
lmbda_l2 = 8e-06
optimizer_name = 'Adam'
opt_lr = 0.001

In [7]:
# save as plain txt
with open(save_dir+"/shared_config_fixed_parameters.txt", "w") as text_file:
    text_file.write("Shared, fixed parameters between Classic and FGN feedforward networks training.\n")
    text_file.write("batch_size {}\n".format(batch_size))
    text_file.write("num_epochs {}\n".format(num_epochs))
    text_file.write("lmbda_l2 {}\n".format(lmbda_l2))
    text_file.write("optimizer_name {}\n".format(optimizer_name))
    text_file.write("opt_lr {}\n".format(opt_lr))

In [8]:
# save as pickle for loading
shared_config_fixed_parameters_dic = {"batch_size":batch_size,
                                       "num_epochs":num_epochs,
                                       "lmbda_l2":lmbda_l2,
                                       "optimizer_name":optimizer_name,
                                       "opt_lr":opt_lr}

with open(save_dir+"/shared_config_fixed_parameters_dic.pckl", 'wb') as f:
    pickle.dump(shared_config_fixed_parameters_dic, f, pickle.HIGHEST_PROTOCOL)

In [9]:
# variable params
# width of the network
hidden_layer_sizes_to_try = [16, 64, 256, 1024]
# depth of the network
# zero added separately
number_of_hidden_layers_to_try = [1,2,3]
drop_ps_to_try = [0.0,0.1,0.4]

In [10]:
# save as plaintext
with open(save_dir+"/shared_config_variable_parameters.txt", "w") as text_file:
    text_file.write("Shared, variable parameters between Classic and FGN feedforward networks training.\n")
    text_file.write("hidden_layer_sizes {}\n".format(str(hidden_layer_sizes_to_try)))
    text_file.write("hidden_layer_numbers {}\n".format(str([]+number_of_hidden_layers_to_try)))
    text_file.write("dropout probs {}".format(str(drop_ps_to_try)))

In [11]:
# list with a bunch of dicts which represent the kwargs for an experiment
exp_params_to_try = []


# define the width and depth of network to try
hidden_layer_params_to_try = []

for (num_layers, layer_sizes) in itertools.product(number_of_hidden_layers_to_try, 
                                                           hidden_layer_sizes_to_try 
                                                           ):
    hidden_layer_params_to_try.append([layer_sizes for _ in range(num_layers)])

# random.shuffle(hidden_layer_params_to_try)

print("hidden_layer_params_to_try")
for x in hidden_layer_params_to_try:
    print(x)

hidden_layer_params_to_try
[16]
[64]
[256]
[1024]
[16, 16]
[64, 64]
[256, 256]
[1024, 1024]
[16, 16, 16]
[64, 64, 64]
[256, 256, 256]
[1024, 1024, 1024]


In [12]:
# define all the experiments to run
exp_params_to_try = []

for (hidden_layer_sizes, drop_p) in itertools.product( 
                                              hidden_layer_params_to_try, 
                                              drop_ps_to_try):
    kwargs = {'hidden_layer_sizes':hidden_layer_sizes,
              'drop_p':drop_p,
             }
    # add to exp to try
    exp_params_to_try.append(kwargs)

# add the network with no hidden layers
exp_params_to_try = [{'hidden_layer_sizes':[], 'drop_p':0.0}] + exp_params_to_try   

In [13]:
exp_params_to_try

[{'drop_p': 0.0, 'hidden_layer_sizes': []},
 {'drop_p': 0.0, 'hidden_layer_sizes': [16]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [16]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [16]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [64]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [64]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [64]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [256]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [256]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [256]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [1024]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [1024]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [1024]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [16, 16]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [16, 16]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [16, 16]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [64, 64]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [64, 64]},
 {'drop_p': 0.4, 'hidden_layer_sizes': [64, 64]},
 {'drop_p': 0.0, 'hidden_layer_sizes': [256, 256]},
 {'drop_p': 0.1, 'hidden_layer_sizes': [256

In [14]:
# save list of all experiments to run
with open(save_dir+"/shared_config_experiments_list.pckl", 'wb') as f:
    pickle.dump(exp_params_to_try, f, pickle.HIGHEST_PROTOCOL)

In [7]:
# FGN variants
# covariance type
covar_types_to_try = ['sphere', 'diag']
# ordinal of gaussian
ordinals_to_try = [0.5, 1.0, 2.0, 5.0]
# pressure on loss to make the gaussians smaller
lmbda_sigmas_to_try = [x*lmbda_l2 for x in [0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.0]]

In [21]:
lmbda_sigmas_to_try

[8e-08,
 8e-07,
 4e-06,
 8e-06,
 1.6e-05,
 7.999999999999999e-05,
 0.0007999999999999999]

In [16]:
# save as plaintext
with open(save_dir+"/fgn_config_variable_parameters.txt", "w") as text_file:
    text_file.write("Variable parameters for the FGN feedforward networks training.\n")
    text_file.write("covar_types_to_try {}\n".format(str(covar_types_to_try)))
    text_file.write("ordinals_to_try {}\n".format(str(ordinals_to_try)))
    text_file.write("lmbda_sigmas_to_try {}".format(str(lmbda_sigmas_to_try)))

In [22]:
# define all the experiments to run
fgn_exp_params_to_try = []

for (lmbda_sigma, covar_type, ordinal) in itertools.product(lmbda_sigmas_to_try,
                                                      covar_types_to_try, 
                                                      ordinals_to_try
                                                      ):
    kwargs = {'lmbda_sigma':lmbda_sigma,
              'covar_type':covar_type,
              'ordinal':ordinal
             }
    # add to exp to try
    fgn_exp_params_to_try.append(kwargs)

In [23]:
fgn_exp_params_to_try

[{'covar_type': 'sphere', 'lmbda_sigma': 8e-08, 'ordinal': 0.5},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-08, 'ordinal': 1.0},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-08, 'ordinal': 2.0},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-08, 'ordinal': 5.0},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-08, 'ordinal': 0.5},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-08, 'ordinal': 1.0},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-08, 'ordinal': 2.0},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-08, 'ordinal': 5.0},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-07, 'ordinal': 0.5},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-07, 'ordinal': 1.0},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-07, 'ordinal': 2.0},
 {'covar_type': 'sphere', 'lmbda_sigma': 8e-07, 'ordinal': 5.0},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-07, 'ordinal': 0.5},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-07, 'ordinal': 1.0},
 {'covar_type': 'diag', 'lmbda_sigma': 8e-07, 'ordinal': 2.0},
 {'covar_type': 'diag', 'lmbda_sigma': 

In [24]:
# save list of all experiments to run
with open(save_dir+"/fgn_config_experiments_list.pckl", 'wb') as f:
    pickle.dump(fgn_exp_params_to_try, f, pickle.HIGHEST_PROTOCOL)